In [1]:
!pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 6.9 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=e456aa3f87ebd5eab05eff37ca49305f925c2c6c77d764328da111873cd8ac63
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [3]:
!git clone https://github.com/PeerapolHan/Bonsai.git

Cloning into 'Bonsai'...
remote: Enumerating objects: 2716, done.
remote: Counting objects: 100% (2716/2716), done.
remote: Compressing objects: 100% (2713/2713), done.
remote: Total 2716 (delta 0), reused 2711 (delta 0), pack-reused 0
Receiving objects: 100% (2716/2716), 18.35 MiB | 9.80 MiB/s, done.


# **upload template.zip(in this folder we have index.html) and file model .pkl**

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Bonsai_classification_VGG16Model.pkl to Bonsai_classification_VGG16Model.pkl
Saving templates.zip to templates.zip


***unzip file***

In [5]:
import zipfile
from zipfile import ZipFile

file_name = '/content/templates.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Extracted Successfully')

Extracted Successfully


# **import the necessary libraries and run the Flask app**

In [6]:
!ngrok authtoken 2Hgdkwoez4L4sld8UYpzC2mn3PZ_7TAK68DLWxd5Ds6tNxCkV

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [9]:
import os
import uuid
import flask
import urllib
from PIL import Image
from flask import Flask , render_template  , request , send_file , session
from tensorflow.keras.preprocessing.image import load_img , img_to_array
from werkzeug.utils import secure_filename
import pickle
import numpy as np
from flask_ngrok import run_with_ngrok
import warnings
warnings.filterwarnings('ignore')

app = Flask(__name__ , static_url_path='')
run_with_ngrok(app)

model = pickle.load(open('Bonsai_classification_VGG16Model.pkl', 'rb'))


ALLOWED_EXT = set(['jpg' , 'jpeg' , 'png' , 'jfif'])
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1] in ALLOWED_EXT

classes = ['Bunjinji','Chokkan','Fukinagashi','Han Kengai','Hokidachi','Moyogi','Seki-joju','Sokan','Yose-ue']

def predict(filename , model):
    img = load_img(filename , target_size = (100 , 100))
    img = img_to_array(img)
    img = img.reshape(1 , 100 ,100 ,3)
    img = img.astype('float32')
    img = img/255.0
    result = model.predict(img)
    dict_result = {}
    for i in range(9):
        dict_result[result[0][i]] = classes[i]
    res = result[0]
    res.sort()
    res = res[::-1]
    prob = res[:3]
    
    prob_result = []
    class_result = []
    for i in range(3):
        prob_result.append((prob[i]*100).round(2))
        class_result.append(dict_result[prob[i]])
    return class_result , prob_result



@app.route('/', methods=['GET'])
def home():
  return render_template('index.html')

@app.route('/upload', methods = ['GET', 'POST'])
def upload_file():
   if request.method == 'POST':
      f = request.files['file']
      if f and allowed_file(f.filename):
        f.save(secure_filename(f.filename))
        img = f.filename
        class_result , prob_result = predict(img , model)
        pred_class1 = class_result[0]
        pred_prob1 = prob_result[0]
        return  render_template('success.html' , img  = img , pred_class1=pred_class1, pred_prob1=pred_prob1)
      return render_template('index.html')



app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4082-35-203-130-182.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 17:32:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 17:32:41] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 17:33:13] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 17:33:15] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 356ms/step


INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 17:33:21] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 17:33:21] "GET /content/aug_126img_36.jpeg HTTP/1.1" 404 -


1/1 [==============================] - 0s 147ms/step


INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 17:33:33] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 17:33:33] "GET /content/aug_126img_36.jpeg HTTP/1.1" 404 -


1/1 [==============================] - 0s 159ms/step


INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 17:33:44] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 17:33:44] "GET /content/aug_136img_59.jpeg HTTP/1.1" 404 -
